In [1]:
#Pandas: Possui inúmeras funções e comandos para importar arquivos, analisar dados, tratar dados, etc.
import pandas as pd # importando a biblioteca pandas

#Numpy: Possui uma série de funções e comandos para trabalharmos com números de forma em geral(formatação, calculos, etc)
import numpy as np # importando a biblioteca numpy

#Time: Possui uma série de funções e comandos para trabalharmos com tempo
import time # importando a biblioteca time

#glob: Possui uma série de funções e comandos para trabalharmos com arquivos
import glob # importando a biblioteca glob

#os: Possui uma série de funções e comandos para trabalharmos com arquivos
import os # importando a biblioteca os

#Warnings: Possui detalhes sobre os avisos e alertas que aparecem, porém podemos utiliza-lo também para que os alertas de
#futuras atualizações e metodos depreciados não sejam exibidos
import warnings
warnings.filterwarnings("ignore") 
tempo_inicial = time.time() # tempo inicial para calcular o tempo de execução do código

from glob import glob # Utilizado para listar arquivos de um diretório

#Comando para exibir todas colunas do arquivo
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Importação dos Dados do CNES 

In [2]:
df_cnes_leitos = pd.read_csv('BASE\CNES\BASE_CNES_LEITOS.csv', sep=';', encoding='latin-1', dtype=str)
df_cnes_leitos.head()

,CO_UNIDADE,CO_CNES,NO_RAZAO_SOCIAL,NO_FANTASIA,TP_UNIDADE,CO_ESTADO_GESTOR,CO_MUNICIPIO_GESTOR,TP_GESTAO,CO_LEITO,CO_TIPO_LEITO,QT_EXIST,QT_SUS
0,4208307575521,7575521,CLINICA JANE REINERT CARRAMILLO,CLINICA JANE REINERT CARRAMILLO,22,42,420830,M,NaN,NaN,NaN,NaN
1,4208307575742,7575742,LE LIS CLINICA,LE LIS CLINICA,22,42,420830,M,NaN,NaN,NaN,NaN
2,4208907566735,7566735,PREFEITURA MUNICIPAL DE JARAGUA DO SUL,UNIDADE BASICA DE SAUDE AMABILE PEDROTTI,2,42,420890,M,NaN,NaN,NaN,NaN
3,4209007574746,7574746,DORE CLINICA DE FISIOTERAPIA E POSTUROLOGIA LTDA,FISIOMAIS,36,42,420900,M,NaN,NaN,NaN,NaN
4,4209107571100,7571100,IMAGEM X RAD TOMO E DIAGNOSTICO ODONTOLOGICO LTDA,IMAGEM X RADIOLOGIA MATRIZ,36,42,420910,M,NaN,NaN,NaN,NaN


In [3]:
df_cnes_habilitacao = pd.read_csv('BASE\CNES\BASE_CNES_HABILITACAO.csv', sep=';', encoding='latin-1', dtype=str)
df_cnes_habilitacao.head()

,CO_UNIDADE,CO_CNES,NO_RAZAO_SOCIAL,NO_FANTASIA,TP_UNIDADE,CO_ESTADO_GESTOR,CO_MUNICIPIO_GESTOR,TP_GESTAO,CO_CODIGO_GRUPO,NO_DESCRICAO_GRUPO
0,4208307575521,7575521,CLINICA JANE REINERT CARRAMILLO,CLINICA JANE REINERT CARRAMILLO,22,42,420830,M,NaN,NaN
1,4208307575742,7575742,LE LIS CLINICA,LE LIS CLINICA,22,42,420830,M,NaN,NaN
2,4208907566735,7566735,PREFEITURA MUNICIPAL DE JARAGUA DO SUL,UNIDADE BASICA DE SAUDE AMABILE PEDROTTI,2,42,420890,M,913.0,ADESAO DO ESTABELECIMENTO AO PROJETO MAIS MÉDICOS
3,4208907566735,7566735,PREFEITURA MUNICIPAL DE JARAGUA DO SUL,UNIDADE BASICA DE SAUDE AMABILE PEDROTTI,2,42,420890,M,7111.0,ESTABELECIMENTO DE SAUDE SEM GERACAO DE CREDIT...
4,4209007574746,7574746,DORE CLINICA DE FISIOTERAPIA E POSTUROLOGIA LTDA,FISIOMAIS,36,42,420900,M,NaN,NaN


In [4]:
df_cnes_servicos = pd.read_csv('BASE\CNES\BASE_CNES_SERVICOS.csv', sep=';', encoding='latin-1', dtype=str)
df_cnes_servicos.head()

,CO_UNIDADE,CO_CNES,NO_RAZAO_SOCIAL,NO_FANTASIA,TP_UNIDADE,CO_ESTADO_GESTOR,CO_MUNICIPIO_GESTOR,TP_GESTAO,CO_SERVICO,CO_CLASSIFICACAO
0,4208307575521,7575521,CLINICA JANE REINERT CARRAMILLO,CLINICA JANE REINERT CARRAMILLO,22,42,420830,M,131.0,2.0
1,4208307575742,7575742,LE LIS CLINICA,LE LIS CLINICA,22,42,420830,M,NaN,NaN
2,4208907566735,7566735,PREFEITURA MUNICIPAL DE JARAGUA DO SUL,UNIDADE BASICA DE SAUDE AMABILE PEDROTTI,2,42,420890,M,112.0,1.0
3,4208907566735,7566735,PREFEITURA MUNICIPAL DE JARAGUA DO SUL,UNIDADE BASICA DE SAUDE AMABILE PEDROTTI,2,42,420890,M,119.0,1.0
4,4208907566735,7566735,PREFEITURA MUNICIPAL DE JARAGUA DO SUL,UNIDADE BASICA DE SAUDE AMABILE PEDROTTI,2,42,420890,M,174.0,1.0


## Importação dos Dados do SIGTAP

## Importação TETO MAC

In [5]:
df_teto = pd.read_csv('BASE\TETO\TETO_MAC.csv', sep=';', encoding='latin-1', dtype=str)
df_teto.head()

,Sigla UF,Código IBGE,Estado / Município,Código Gestão,Descrição Gestão,Teto Financeiro MAC - Valores Anuais (R$)
0,AC,120001,ACRELÂNDIA,1,Gestão Municipal,372836.91
1,AC,120005,ASSIS BRASIL,1,Gestão Municipal,0.0
2,AC,120010,BRASILÉIA,1,Gestão Municipal,339660.0
3,AC,120013,BUJARI,1,Gestão Municipal,0.0
4,AC,120017,CAPIXABA,1,Gestão Municipal,339660.0


## Importação da PLANILHA  

In [6]:
df_planilha_aba1 = pd.read_excel('PLANILHA\BASE.xlsx', sheet_name='Ident. Fila na UF') # Lê o arquivo excel
df_planilha_aba1.head() # Exibe as 5 primeiras linhas do arquivo

,PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - FILA DE ESPERA,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Planilha protegida devido as fórmulas existent...,NaN,NaN,NaN,NaN,NaN,Maranhão,VRS 4
1,CÓDIGO DO PROCEDIMENTO NO SIGTAP,PROCEDIMENTO CIRÚRGICO,QUANTIDADE DE SOLICITAÇÕES NA FILA ATÉ DIA 31/...,Redução do Tamanho da Fila (%),Prazo para reduzir o % proposto (em meses),Qtde de cirurgias a serem feitas no prazo pact...,SQ (CODIGO Interno,NaN
2,401020010,ENXERTO COMPOSTO,3,0.593,10,1.779,100001,NaN
3,401020029,ENXERTO DERMO-EPIDERMICO,4,0.593,10,2.372,100002,NaN
4,401020045,"EXCISAO E ENXERTO DE PELE (HEMANGIOMA, NEVUS O...",365,0.593,10,216.445,100003,NaN


In [7]:
df_planilha_aba1.rename(columns={'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - FILA DE ESPERA':'COD_PROCEDIMENTO','Unnamed: 1':'DESC_PROCEDIMENTO','Unnamed: 2':'QUANT_FILA', 
                                 'Unnamed: 3':'PERC_REDUCAO', 'Unnamed: 4':'TEMPO_MESES', 'Unnamed: 5':'QUANT_REDUCAO', 'Unnamed: 6':'LINHA'}, inplace=True) 
                                # Renomeia a coluna 'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - FILA DE ESPERA' para 'UF'
df_planilha_aba1.drop(0, inplace=True) # Remove a primeira linha do arquivo
df_planilha_aba1.drop(1, inplace=True) # Remove a segunda linha do arquivo
df_planilha_aba1.drop(df_planilha_aba1.tail(1).index,inplace=True) # Removendo a última linha do arquivo
df_planilha_aba1.dropna(subset=['COD_PROCEDIMENTO'], inplace=True) # Removendo linhas com valores nulos
df_planilha_aba1.drop('Unnamed: 7', axis=1, inplace=True) # Removendo coluna 'Unnamed: 7'
df_planilha_aba1.head() # Exibe as 5 primeiras linhas do arquivo

,COD_PROCEDIMENTO,DESC_PROCEDIMENTO,QUANT_FILA,PERC_REDUCAO,TEMPO_MESES,QUANT_REDUCAO,LINHA
2,401020010,ENXERTO COMPOSTO,3,0.593,10,1.779,100001
3,401020029,ENXERTO DERMO-EPIDERMICO,4,0.593,10,2.372,100002
4,401020045,"EXCISAO E ENXERTO DE PELE (HEMANGIOMA, NEVUS O...",365,0.593,10,216.445,100003
5,401020061,EXERESE DE CISTO BRANQUIAL,55,0.593,10,32.615,100004
6,401020070,EXERESE DE CISTO DERMOIDE,1103,0.593,10,654.079,100005


In [8]:
df_planilha_aba1['COD_PROCEDIMENTO'] = df_planilha_aba1['COD_PROCEDIMENTO'].astype(float) # Converte a coluna 'COD_PROCEDIMENTO' para string
df_planilha_aba1.info() # Exibe informações sobre o arquivo

<class 'pandas.core.frame.DataFrame'>
Int64Index: 483 entries, 2 to 484
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   COD_PROCEDIMENTO   483 non-null    float64
 1   DESC_PROCEDIMENTO  483 non-null    object 
 2   QUANT_FILA         483 non-null    object 
 3   PERC_REDUCAO       483 non-null    object 
 4   TEMPO_MESES        483 non-null    object 
 5   QUANT_REDUCAO      483 non-null    object 
 6   LINHA              483 non-null    object 
dtypes: float64(1), object(6)
memory usage: 30.2+ KB


In [9]:
# Verifica se há valores duplicados na coluna "COD_PROCEDIMENTO"
df_planilha_aba1['DUPLICADO'] = df_planilha_aba1['COD_PROCEDIMENTO'].duplicated(keep=False)

# Cria uma nova coluna com as mensagens correspondentes
df_planilha_aba1['M_DUPLICADO'] = np.where(df_planilha_aba1['DUPLICADO'], 'procedimento duplicado', 'procedimento OK')

# Exibe as 5 primeiras linhas do arquivo com a nova coluna
df_planilha_aba1.head()

,COD_PROCEDIMENTO,DESC_PROCEDIMENTO,QUANT_FILA,PERC_REDUCAO,TEMPO_MESES,QUANT_REDUCAO,LINHA,DUPLICADO,M_DUPLICADO
2,401020010.0,ENXERTO COMPOSTO,3,0.593,10,1.779,100001,False,procedimento OK
3,401020029.0,ENXERTO DERMO-EPIDERMICO,4,0.593,10,2.372,100002,False,procedimento OK
4,401020045.0,"EXCISAO E ENXERTO DE PELE (HEMANGIOMA, NEVUS O...",365,0.593,10,216.445,100003,False,procedimento OK
5,401020061.0,EXERESE DE CISTO BRANQUIAL,55,0.593,10,32.615,100004,False,procedimento OK
6,401020070.0,EXERESE DE CISTO DERMOIDE,1103,0.593,10,654.079,100005,False,procedimento OK


In [10]:
# Adicionando nova coluna 'QUANT_ZERO' com a mensagem correspondente
df_planilha_aba1['QUANT_ZERO'] = np.where(df_planilha_aba1['QUANT_REDUCAO'] < 1, 'quant. Menor que 1', 'quant. OK!')

# Exibindo as primeiras linhas do dataframe com a nova coluna
df_planilha_aba1.head()


,COD_PROCEDIMENTO,DESC_PROCEDIMENTO,QUANT_FILA,PERC_REDUCAO,TEMPO_MESES,QUANT_REDUCAO,LINHA,DUPLICADO,M_DUPLICADO,QUANT_ZERO
2,401020010.0,ENXERTO COMPOSTO,3,0.593,10,1.779,100001,False,procedimento OK,quant. OK!
3,401020029.0,ENXERTO DERMO-EPIDERMICO,4,0.593,10,2.372,100002,False,procedimento OK,quant. OK!
4,401020045.0,"EXCISAO E ENXERTO DE PELE (HEMANGIOMA, NEVUS O...",365,0.593,10,216.445,100003,False,procedimento OK,quant. OK!
5,401020061.0,EXERESE DE CISTO BRANQUIAL,55,0.593,10,32.615,100004,False,procedimento OK,quant. OK!
6,401020070.0,EXERESE DE CISTO DERMOIDE,1103,0.593,10,654.079,100005,False,procedimento OK,quant. OK!


In [11]:
df_planilha_aba2 = pd.read_excel('PLANILHA\BASE.xlsx', sheet_name='Ident. CNES e Proced.', dtype=str) # Lê o arquivo excel
df_planilha_aba2.head() # Exibe as 5 primeiras linhas do arquivo

,PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - CNES,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,Planilha protegida devido as fórmulas existent...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VRS 4,Maranhão
1,CNES,ESTABELECIMENTO,CÓDIGO DO PROCEDIMENTO PRINCIPAL NO SIGTAP,PROCEDIMENTO CIRÚRGICO (PRINCIPAL),COMPLEMENTO COM RECURSO FEDERAL DO PROCEDIMENT...,GESTÃO DO SERVIÇO,CODIGO DA NATUREZA JURÍDICA,NATUREZA JURÍDICA,POSSUI CONTRATO COM A SECRETARIA DE SAÚDE ?,Identificação do Nome do Estabelecimento que n...,SQ (CODIGO Interno
2,2307049,HOSPITAL MUNICIPAL SANTO ANTONIO,405050372,FACOEMULSIFICACAO C/ IMPLANTE DE LENTE INTRA-O...,NaN,MUNICIPAL,1244,MUNICIPIO,NaN,NaN,200001
3,2307898,HOSPITAL MUNICIPAL ZUZA COELHO,405050372,FACOEMULSIFICACAO C/ IMPLANTE DE LENTE INTRA-O...,NaN,MUNICIPAL,1244,MUNICIPIO,NaN,NaN,200002
4,2307898,HOSPITAL MUNICIPAL ZUZA COELHO,407020284,HEMORROIDECTOMIA,NaN,MUNICIPAL,1244,MUNICIPIO,NaN,NaN,200003


In [12]:
df_planilha_aba2.rename(columns={'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - CNES':'CNES','Unnamed: 1':'ESTABELECIMENTO','Unnamed: 2':'COD_PROCEDIMENTO', 
                                 'Unnamed: 3':'DES_PROCEDIMENTO', 'Unnamed: 4':'COMPL_RECURSO_FEDERAL', 'Unnamed: 5':'GESTAO', 'Unnamed: 6':'COD_NATUREZA','Unnamed: 7':'NATUREZA',
                                 'Unnamed: 8':'POSSUI_CONTRATO','Unnamed: 9':'IDENTIFICACAO','Unnamed: 10':'LINHA'}, inplace=True) 
                                # Renomeia a coluna 'PLANO ESTADUAL DE REDUÇÃO DE FILAS DE ESPERA EM CIRURGIAS ELETIVAS - FILA DE ESPERA' para 'UF'
df_planilha_aba2.drop(0, inplace=True) # Remove a primeira linha do arquivo
df_planilha_aba2.drop(1, inplace=True) # Remove a primeira linha do arquivo
df_planilha_aba2.drop(df_planilha_aba2.tail(1).index,inplace=True) # Removendo a última linha do arquivo
df_planilha_aba2.dropna(subset=['CNES'], inplace=True) # Removendo linhas com valores nulos
df_planilha_aba2.drop('IDENTIFICACAO', axis=1, inplace=True) # Removendo coluna 'IDENTIFICACAO'
df_planilha_aba2.head() # Exibe as 5 primeiras linhas do arquivo

,CNES,ESTABELECIMENTO,COD_PROCEDIMENTO,DES_PROCEDIMENTO,COMPL_RECURSO_FEDERAL,GESTAO,COD_NATUREZA,NATUREZA,POSSUI_CONTRATO,LINHA
2,2307049,HOSPITAL MUNICIPAL SANTO ANTONIO,405050372,FACOEMULSIFICACAO C/ IMPLANTE DE LENTE INTRA-O...,NaN,MUNICIPAL,1244,MUNICIPIO,NaN,200001
3,2307898,HOSPITAL MUNICIPAL ZUZA COELHO,405050372,FACOEMULSIFICACAO C/ IMPLANTE DE LENTE INTRA-O...,NaN,MUNICIPAL,1244,MUNICIPIO,NaN,200002
4,2307898,HOSPITAL MUNICIPAL ZUZA COELHO,407020284,HEMORROIDECTOMIA,NaN,MUNICIPAL,1244,MUNICIPIO,NaN,200003
5,2307898,HOSPITAL MUNICIPAL ZUZA COELHO,407030026,COLECISTECTOMIA,NaN,MUNICIPAL,1244,MUNICIPIO,NaN,200004
6,2307898,HOSPITAL MUNICIPAL ZUZA COELHO,407040099,HERNIOPLASTIA INGUINAL (BILATERAL),NaN,MUNICIPAL,1244,MUNICIPIO,NaN,200005


In [13]:
df_planilha_aba2['COD_PROCEDIMENTO'] = df_planilha_aba2['COD_PROCEDIMENTO'].astype(float) # Converte a coluna 'COD_PROCEDIMENTO' para string
df_planilha_aba2.info() # Exibe informações sobre o arquivo

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2622 entries, 2 to 2623
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CNES                   2622 non-null   object 
 1   ESTABELECIMENTO        2562 non-null   object 
 2   COD_PROCEDIMENTO       2622 non-null   float64
 3   DES_PROCEDIMENTO       2622 non-null   object 
 4   COMPL_RECURSO_FEDERAL  0 non-null      object 
 5   GESTAO                 2622 non-null   object 
 6   COD_NATUREZA           2562 non-null   object 
 7   NATUREZA               2562 non-null   object 
 8   POSSUI_CONTRATO        138 non-null    object 
 9   LINHA                  2622 non-null   object 
dtypes: float64(1), object(9)
memory usage: 225.3+ KB


In [14]:
# Idenfica se o procedimento informado na aba anterior está na aba atual
df_planilha_aba2['PROC_INFORMADO'] = np.where(df_planilha_aba2['COD_PROCEDIMENTO'].isin(df_planilha_aba1['COD_PROCEDIMENTO']), 'SIM','NÃO')                                     

In [15]:
df_planilha_aba2.head()

,CNES,ESTABELECIMENTO,COD_PROCEDIMENTO,DES_PROCEDIMENTO,COMPL_RECURSO_FEDERAL,GESTAO,COD_NATUREZA,NATUREZA,POSSUI_CONTRATO,LINHA,PROC_INFORMADO
2,2307049,HOSPITAL MUNICIPAL SANTO ANTONIO,405050372.0,FACOEMULSIFICACAO C/ IMPLANTE DE LENTE INTRA-O...,NaN,MUNICIPAL,1244,MUNICIPIO,NaN,200001,SIM
3,2307898,HOSPITAL MUNICIPAL ZUZA COELHO,405050372.0,FACOEMULSIFICACAO C/ IMPLANTE DE LENTE INTRA-O...,NaN,MUNICIPAL,1244,MUNICIPIO,NaN,200002,SIM
4,2307898,HOSPITAL MUNICIPAL ZUZA COELHO,407020284.0,HEMORROIDECTOMIA,NaN,MUNICIPAL,1244,MUNICIPIO,NaN,200003,SIM
5,2307898,HOSPITAL MUNICIPAL ZUZA COELHO,407030026.0,COLECISTECTOMIA,NaN,MUNICIPAL,1244,MUNICIPIO,NaN,200004,SIM
6,2307898,HOSPITAL MUNICIPAL ZUZA COELHO,407040099.0,HERNIOPLASTIA INGUINAL (BILATERAL),NaN,MUNICIPAL,1244,MUNICIPIO,NaN,200005,SIM


In [16]:
# Verifica se o procedimento informado possui prestador informado
df_planilha_aba1['POSSUI_PRESTADOR'] = np.where(df_planilha_aba1['COD_PROCEDIMENTO'].isin(df_planilha_aba2['COD_PROCEDIMENTO']),'SIM','NÃO')

In [17]:
df_planilha_aba1.head()

,COD_PROCEDIMENTO,DESC_PROCEDIMENTO,QUANT_FILA,PERC_REDUCAO,TEMPO_MESES,QUANT_REDUCAO,LINHA,DUPLICADO,M_DUPLICADO,QUANT_ZERO,POSSUI_PRESTADOR
2,401020010.0,ENXERTO COMPOSTO,3,0.593,10,1.779,100001,False,procedimento OK,quant. OK!,SIM
3,401020029.0,ENXERTO DERMO-EPIDERMICO,4,0.593,10,2.372,100002,False,procedimento OK,quant. OK!,SIM
4,401020045.0,"EXCISAO E ENXERTO DE PELE (HEMANGIOMA, NEVUS O...",365,0.593,10,216.445,100003,False,procedimento OK,quant. OK!,SIM
5,401020061.0,EXERESE DE CISTO BRANQUIAL,55,0.593,10,32.615,100004,False,procedimento OK,quant. OK!,SIM
6,401020070.0,EXERESE DE CISTO DERMOIDE,1103,0.593,10,654.079,100005,False,procedimento OK,quant. OK!,SIM


In [18]:
# Verificação de procedimento duplicado
if df_planilha_aba1['M_DUPLICADO'].str.contains('procedimento duplicado').any():
    print("Existem procedimentos na Fila, duplicado;")
    procedimento_duplicado = True
else:
    procedimento_duplicado = False

# Verificação de quantidade menor que 1
if df_planilha_aba1['QUANT_ZERO'].str.contains('quant. Menor que 1').any():
    print("Existem procedimentos na Fila, cuja quantidade de cirurgia ficou inferior a 1 procedimento (PACIENTE);")
    quant_menor_1 = True
else:
    quant_menor_1 = False

# Verificação de procedimentos sem prestador
if df_planilha_aba1['POSSUI_PRESTADOR'].str.contains('NÃO').any():
    print("Existem procedimentos na Fila, cuja não existe prestador;")
    sem_prestador = True
else:
    sem_prestador = False


In [19]:
# Verifica se os valores da coluna "COD_PROCEDIMENTO" do DataFrame "df_planilha_aba2" estão contidos na coluna "COD_PROCEDIMENTO" do DataFrame "df_planilha_aba1"
df_planilha_aba2['PROC_INFORMADO'] = np.where(df_planilha_aba2['COD_PROCEDIMENTO'].isin(df_planilha_aba1['COD_PROCEDIMENTO']), 'proc. Informado na Ident. Fila na UF', 'proc. não informado na aba anterior')
df_planilha_aba2.head()

,CNES,ESTABELECIMENTO,COD_PROCEDIMENTO,DES_PROCEDIMENTO,COMPL_RECURSO_FEDERAL,GESTAO,COD_NATUREZA,NATUREZA,POSSUI_CONTRATO,LINHA,PROC_INFORMADO
2,2307049,HOSPITAL MUNICIPAL SANTO ANTONIO,405050372.0,FACOEMULSIFICACAO C/ IMPLANTE DE LENTE INTRA-O...,NaN,MUNICIPAL,1244,MUNICIPIO,NaN,200001,proc. Informado na Ident. Fila na UF
3,2307898,HOSPITAL MUNICIPAL ZUZA COELHO,405050372.0,FACOEMULSIFICACAO C/ IMPLANTE DE LENTE INTRA-O...,NaN,MUNICIPAL,1244,MUNICIPIO,NaN,200002,proc. Informado na Ident. Fila na UF
4,2307898,HOSPITAL MUNICIPAL ZUZA COELHO,407020284.0,HEMORROIDECTOMIA,NaN,MUNICIPAL,1244,MUNICIPIO,NaN,200003,proc. Informado na Ident. Fila na UF
5,2307898,HOSPITAL MUNICIPAL ZUZA COELHO,407030026.0,COLECISTECTOMIA,NaN,MUNICIPAL,1244,MUNICIPIO,NaN,200004,proc. Informado na Ident. Fila na UF
6,2307898,HOSPITAL MUNICIPAL ZUZA COELHO,407040099.0,HERNIOPLASTIA INGUINAL (BILATERAL),NaN,MUNICIPAL,1244,MUNICIPIO,NaN,200005,proc. Informado na Ident. Fila na UF


In [20]:
merged = df_planilha_aba2.merge(df_planilha_aba1[['COD_PROCEDIMENTO']], on='COD_PROCEDIMENTO', how='left', indicator='PROC_INFORMADO')
df_planilha_aba2['PROC_INFORMADO'] = np.where(merged['PROC_INFORMADO'] == 'both', 'proc. Informado na Ident. Fila na UF', 'proc. não informado na aba anterior')



ValueError: Cannot use name of an existing column for indicator column

In [ ]:
df_planilha_aba2.head(315)

In [ ]:
df_planilha_aba1.head(315)